## Need to be running a python  version that works with outlier_utils

[Resource](https://www.alfredo.motta.name/create-isolated-jupyter-ipython-kernels-with-pyenv-and-virtualenv/) for having multiple versions of python + Jupyter. The colab version is 3.6.9 but not an option in pyenv so went with 3.6.8

In [ ]:
from platform import python_version

print(python_version())

## Update path_j based on user & install packages

In [ ]:
#google drive authorize AND googlesheets4 authorize-- must create a project under google API and then download credentials
#see authentication section in the following article
#https://medium.com/@bretcameron/how-to-use-the-google-drive-api-with-javascript-57a6cc9e5262
#Note: need to open a service account and extract .json credentials in the API console, approve Google Drive API, then share the service account email address with the sheet of interest
path_j='/Users/owner/Desktop/Berkeley_Work/Projects/2017_San_Diego/June 2018_Data_Analysis/access-via-jupiter-notebook-d18b3ab0993b.json' #lauren's


#need  to do the following  in the activated virtual environment
# !pip install gspread
# !pip install oauth2client
# !pip install outliers
# !pip install plotnine
# !pip install scikit-misc
# !pip install outlier_utils
# !pip install pandas
# !pip  install numpy
# !pip install sklearn

In [3]:
import sys
import os
from os import path
from datetime import date
import math
import warnings
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from plotnine import *
from scipy import stats
from scipy.stats import linregress
import gspread
from mizani.formatters import scientific_format

%matplotlib inline

import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name(path_j, scope)
#Authorize credentials and open master sampling table using its URL
gc = gspread.authorize(credentials)
# routine= gc.open_by_url('https://docs.google.com/spreadsheets/d/163xXZnJjh0LdryxrDhuYxgzHB_IY7Jaew3DpYzoNs9Q/edit#gid=1255279898')
# qpcr=gc.open_by_url('https://docs.google.com/spreadsheets/d/1TzZcB2p55BuG5FbXAA44EFoe8BdRapGyDaVR4hx9NXU/edit#gid=1689469809')
import skmisc
import outliers

# read in custom modules
from read_gsheets import * 
from reprocess_qpcr import *
from calculations import *
from qa_qc import *

#urls
samples_url = 'https://docs.google.com/spreadsheets/d/163xXZnJjh0LdryxrDhuYxgzHB_IY7Jaew3DpYzoNs9Q'
qpcr_url = 'https://docs.google.com/spreadsheets/d/1TzZcB2p55BuG5FbXAA44EFoe8BdRapGyDaVR4hx9NXU'

#sheets
rna_tab = 'sample_inventory'
ww_tab='site_lookup'
facility_lookup='site_lookup'
qpcr_results_tab = 'QuantStudio_raw_data'
qpcr_plates_tab = 'Plate_info'

NameError: name 'credentials' is not defined

In [ ]:
# read in data
sample_data = read_sample_data(gc, samples_url, rna_tab, facility_lookup)
qpcr_raw = read_qpcr_data(gc, qpcr_url, qpcr_results_tab, qpcr_plates_tab)
qpcr_processed, std_curve_df, raw_outliers_flagged_df = process_qpcr_raw(qpcr_raw, 'grubbs_only')

# merge with sample data and remove NTCs and Xeno from averaged data
qpcr_averaged = qpcr_processed.merge(sample_data, how='left', left_on='Sample', right_on='sample_id')
qpcr_averaged = qpcr_averaged[(qpcr_averaged.Sample != 'NTC') &
                              (qpcr_averaged.Target != 'Xeno')]
# calculations
qpcr_averaged['gc_per_L'] = calculate_gc_per_l(qpcr_averaged) # get gc/L
qpcr_normd = normalize_to_pmmov(qpcr_averaged)
#qpcr_normd = get_GFP_recovery(qpcr_normd) # this converts dates to numeric somehow

## Filter out research batches
qpcr_normd = qpcr_normd[~(qpcr_normd['batch'].str.contains("B", na=True)) &
                        ~(qpcr_normd['batch'].str.contains("V", na=True))].copy()

# make 'PBS_result' column and remove extraction controls from main dataset
qpcr_normd = get_extraction_control(qpcr_normd)

In [ ]:
xeno_inhib_full, xeno_control=xeno_inhibition_test(raw_outliers_flagged_df)

xeno_inhib=xeno_inhib_full.merge(sample_data, left_on='Sample', right_on='sample_id',  how='left').copy()
inhibited=xeno_inhib[xeno_inhib.dCt>1].Sample.unique()
not_inhibited=xeno_inhib[xeno_inhib.dCt<=1].Sample.unique()

qpcr_normd["is_inhibited"]='unknown'
qpcr_normd.loc[qpcr_normd.Sample.isin(inhibited),"is_inhibited"]="Yes"
qpcr_normd.loc[qpcr_normd.Sample.isin(not_inhibited),"is_inhibited"]="No"

In [ ]:
qpcr_averaged_merged.head()